# Exercise 1
plot following Green's functions of the single-orbital fermionic Hubbard atom at half filling,
$$
G_{\sigma\sigma'}(z) = \frac{1}{2}\left(\frac{1}{z-U/2} + \frac{1}{z+U/2} \right)\delta_{\sigma\sigma'}
$$

$$
G_{\sigma\sigma'} = \frac{-1}{2}\left(\frac{e^{-\tau U/2}}{1+e^{-\beta U/2}} + \frac{e^{\tau U/2}}{1+e^{\beta U/2}} \right)
$$

In [217]:
using PyPlot
using LinearAlgebra
using BenchmarkTools
using PyCall 

In [218]:
function G(τ,U,β)
	- (exp(-τ*U/2)/(1+exp(-β*U/2)) + exp(τ*U/2)/(1+exp(β*U/2))) / 2
end

G (generic function with 2 methods)

In [219]:
U = 1
β = 300
τs = LinRange(0,300,600)
plt.figure()
plt.plot(τs,G.(τs,U,β))
plt.title("G(τ) computed for the Hubbard atom withU= 1 and β= 300.")
plt.xlabel("τ")
plt.ylabel("G(τ)")
plt.savefig("exercise_1_1.pdf")

In [220]:
plt.figure()
plt.plot(τs,abs.(G.(τs,U,β)))
plt.title("G(τ) computed for the Hubbard atom withU= 1 and β= 300.")
plt.yscale("log")
plt.xlabel("τ")
plt.ylabel("|G|")
plt.savefig("exercise_1_2.pdf")

In [221]:
function G(z,U)
	(1/(z-U/2) + 1/(z+U/2))/2
end

U  = 1.0
νs = LinRange(-4,4,100)
plt.figure()
plt.plot(νs,imag.(G.(im.*νs,U)),marker="o",label="Im G(iν)")
plt.plot(νs,1 ./ νs,label="1/ν")
plt.title("G(iν) computed for the Hubbard atom withU= 1 and β= 300.")
plt.xlabel("ν")
plt.ylabel("Im G(iν)")
plt.ylim(-1.5,1.5)
plt.legend(loc="upper right")
plt.savefig("exercise_1_3.pdf")

# Exercise 2
For the Hubbard atom, Green's function reads
$$
G_{\sigma\sigma'}(z) = \frac{1}{2}\left(\frac{1}{z-U/2} + \frac{1}{z+U/2} \right)\delta_{\sigma\sigma'}.
$$
Using this formula
$$
T\sum_{\nu}\frac{e^{-i\nu T}}{i\nu-\epsilon} = -\frac{e^{-\epsilon\tau}}{1+e^{-\beta\epsilon}},
$$
we obtain 
$$
G_{\sigma\sigma'}(\tau) 
= T\sum_n e^{-i\omega_n\tau}G(i\omega_n)
= \frac{T}{2}\sum_n \left[\frac{e^{-i\omega_n\tau}}{i\omega_n-U/2} + \frac{e^{-i\omega_n\tau}}{i\omega_n+U/2}\right]\delta_{\sigma\sigma'}
= -\frac{1}{2}\left(\frac{e^{\tau U/2}}{1+e^{-\beta U/2}} + \frac{e^{\tau U/2}}{1+e^{\beta U/2}}\right)\delta_{\sigma\sigma'}
$$

# Exercise 3
Please numerically check how $G(\tau)$ of the Hubbard atom behaves near $\tau= 0$ if we truncate the summation in the following equation at some frequency.
$$
T\sum_{\nu}\frac{e^{-i\nu T}}{i\nu-\epsilon} = -\frac{e^{-\epsilon\tau}}{1+e^{-\beta\epsilon}}.
$$
$G(\tau)$ reads
$$
G(\tau) = T\sum_n e^{-i\omega_n\tau}G(i\omega_n)
$$
where $\omega_n = (2n+1)\pi T.$

In [222]:
#ω_n = (2n+1)πT
U = 1.
β = 300.
T = 1/β
num_tau = 1000
τs = LinRange(0,β,num_tau)
num_omega = 200
ω_n = [(2n+1)*π*T for n in -num_omega:num_omega]

function naive_inverse_Giv_forloop(num_tau,ω_n,U,T,τs)
	G_tau = zeros(Complex,num_tau) 

    for i_tau in 1:num_tau
	    for j_omega in 1:length(ω_n)
		    G_tau[i_tau] += T * exp(- im * ω_n[j_omega] * τs[i_tau]) * G(im * ω_n[j_omega],U)
	    end
    end
	G_tau
end

G_tau = naive_inverse_Giv_forloop(num_tau,ω_n,U,T,τs)
plt.figure()
plt.plot(τs,real.(G_tau))
plt.plot(τs,G.(τs,U,β))
plt.xlim(-0.5,10)
plt.xlabel("τ")
plt.ylabel("G(τ)")
plt.title("Naive inverse transform of G(iν) for the Hubbard atom with U=1 and β=300")
plt.savefig("Exercise_3.pdf")

In [223]:
# 行列積での計算
#ω_n = (2n+1)πT
U = 1.
β = 300.
T = 1/β
num_tau = 1000
τs = collect(LinRange(0,β,num_tau))
num_omega = 200
ω_n = [(2n+1)*π*T for n in -num_omega:num_omega]

function naive_inverse_Giv_matprod(τs,ω_n,U,T)
	G_iν = G.(im .* ω_n,U)
    exp_tau_iomega = exp.(τs * transpose(-im .* ω_n))
    G_tau = T .* (exp_tau_iomega * G_iν)
end

G_tau = naive_inverse_Giv_matprod(τs,ω_n,U,T)
plt.figure()
plt.plot(τs,real.(G_tau))
plt.plot(τs,G.(τs,U,β))
plt.xlim(-0.5,10)
plt.xlabel("τ")
plt.ylabel("G(τ)")
plt.title("Naive inverse transform of G(iν) for the Hubbard atom with U=1 and β=300")
plt.savefig("Exercise_3.pdf")

In [224]:
G_tau_1 = naive_inverse_Giv_forloop(num_tau,ω_n,U,T,τs)
G_tau_2 = naive_inverse_Giv_matprod(τs,ω_n,U,T)
plt.figure()
plt.plot(τs,real.(G_tau_1),label="with for loop",marker="o")
plt.plot(τs,real.(G_tau_2),label="with matrix product")
plt.xlim(0,10)
plt.xlabel("τ")
plt.ylabel("G(τ)")
plt.legend(loc="upper right")
plt.savefig("check_exercise_3.pdf")

In [225]:
# Benchmark
@benchmark naive_inverse_Giv_forloop(num_tau,ω_n,U,T,τs)

BenchmarkTools.Trial: 
  memory estimate:  24.48 MiB
  allocs estimate:  802001
  --------------
  minimum time:     45.966 ms (0.00% GC)
  median time:      60.967 ms (0.00% GC)
  mean time:        86.382 ms (5.87% GC)
  maximum time:     482.338 ms (4.43% GC)
  --------------
  samples:          59
  evals/sample:     1

In [226]:
@benchmark naive_inverse_Giv_matprod(τs,ω_n,U,T)

BenchmarkTools.Trial: 
  memory estimate:  12.28 MiB
  allocs estimate:  8
  --------------
  minimum time:     9.305 ms (0.00% GC)
  median time:      11.661 ms (0.00% GC)
  mean time:        17.008 ms (10.94% GC)
  maximum time:     173.700 ms (0.00% GC)
  --------------
  samples:          294
  evals/sample:     1

# Exercise 4

In [227]:
using Interpolations,Dierckx

In [228]:
# linear and cubic spline interpolations of G(τ)
U = 1.
β = 300.
num_tau = 100

τs = collect(range(0.0,β,length=num_tau))
G_tau_abs = abs.(G.(τs,U,β))
τs_new = collect(range(0,β,length=10*num_tau))

# Linear interpolation
G_tau_linear = LinearInterpolation(τs,G_tau_abs)(τs_new)

# Cubic spline interpolation
G_tau_cubic_spline = Spline1D(τs,G_tau_abs)(τs_new)

plt.figure()
plt.plot(τs,G_tau_abs,marker="o",label="Data points",color="black")
plt.plot(τs_new,abs.(G.(τs_new,U,β)),label="Exact",color="black")
plt.plot(τs_new,G_tau_linear,ls="--",label="Linear",color="red")
plt.plot(τs_new,G_tau_cubic_spline,ls="-.",label="Cubic spline",color="blue")
plt.xlabel("τ")
plt.ylabel("|G(τ)|")
plt.xlim(0,20)
plt.legend(loc="upper right")
plt.tight_layout()
plt.savefig("Exercise_4_1.pdf")

In [229]:
plt.figure()
plt.semilogy(τs_new,abs.(G_tau_linear       - G.(τs_new,U,β)), ls="--",label="Linear",      color="red")
plt.semilogy(τs_new,abs.(G_tau_cubic_spline - G.(τs_new,U,β)), ls="-.",label="Cubic spline",color="blue")
plt.xlabel("τ")
plt.ylabel("Absolute Error")
plt.xlim(0,20)
plt.ylim(1e-5,1)
plt.legend(loc="upper right")
plt.tight_layout()
plt.savefig("Exercise_4_2.pdf")

# Exercise 4.5
Implement Fourier transform using Gauss quadrature.
$$
G(i\nu) = \int_0^{\beta}\mathrm{d}\tau e^{i\nu_n\tau}G(\tau) \sim \sum_{i=1}^n \tilde{\omega_i}e^{i\nu\tau_i}G(\tau_i)
$$
where $\tau_i = (x_i+1)\beta/2$ and $\tilde{\omega_i} = \beta\omega_i/2$

In [230]:
using FastGaussQuadrature,SpecialPolynomials

In [231]:
U = 1.
β = 300.
deg = 1000
x, w = gausslegendre(deg)
τs_leggauss = (x .+ 1)*β/2
ws_leggauss = (β/2) * w

ω_max = 2000
ω_n = [(2i+1)*π/β for i in -ω_max:ω_max][1:20:end]
G_tau = G.(τs_leggauss,U,β)
exp_iomega_tau = exp.((im .* ω_n) * transpose(τs_leggauss))

G_iν_leggauss = exp_iomega_tau * (ws_leggauss .* G_tau)
G_iν_exact    = G.(im .* ω_n,U)
 
plt.figure()
plt.plot(ω_n,imag.(G_iν_leggauss),ls="-",marker="x",label="Gauss-Legendre")
plt.plot(ω_n,imag.(G_iν_exact),   ls="-",marker="o",label="Exact")
plt.xlim(-40,40)
plt.ylim(-1,1)
plt.xlabel("ν")
plt.ylabel("Im G(iν)")
plt.legend(loc="upper right")
plt.savefig("Exercise_4.5_1.pdf")

In [232]:
plt.figure()
plt.semilogy(ω_n,abs.(G_iν_leggauss .- G_iν_exact),ls="-",label="Absolute Error")
plt.semilogy(ω_n,abs.((G_iν_leggauss .- G_iν_exact) ./ G_iν_exact),ls="--",label="Relative Error")
plt.xlabel("ν")
plt.ylabel("Error")
plt.legend(loc="upper center")
plt.savefig("Exercise_4.5_2.pdf")


# Exercise 5


# Exercise 6
Implement following equations for the Hubbard atom 
$$
\tilde{G}(i\nu) = G(i\nu) - \frac{c^0}{i\nu}, 
$$
$$
G(\tau) \sim \sum_{n=-n_{\mathrm{max}}}^{n_{\mathrm{max}}}e^{i\nu_n\tau}\tilde{G}(i\nu) - \frac{c^0}{2}
$$
and comform that the discontinuity at $\tau = 0$ is correctly reproduced.

In [233]:
# 行列積での計算
#ω_n = (2n+1)πT
U = 1.
β = 300.
T = 1/β
num_tau = 1000
τs = collect(range(0,β,length=num_tau))
num_omega = 200
ω_n = [(2n+1)*π*T for n in -num_omega:num_omega]

function trickey_inverse_Giv_matprod(τs,ω_n,U,T)
	G_iν_tilde = G.(im .* ω_n,U) .- (1 ./ (im .* ω_n))
    exp_tau_iomega = exp.(τs * transpose(-im .* ω_n))
    G_tau = T .* (exp_tau_iomega * G_iν_tilde) .- 1/2
end

G_tau = trickey_inverse_Giv_matprod(τs,ω_n,U,T)
plt.figure()
plt.plot(τs,real.(G_tau),marker="o",label="tricky inverse of G(iν)")
plt.plot(τs,G.(τs,U,β),label="Exact")
plt.xlim(-0.5,10)
plt.xlabel("τ")
plt.ylabel("G(τ)")
plt.title("trickey inverse transform of G(iν) for the Hubbard atom with U=1 and β=300")
plt.legend(loc="upper left")
plt.savefig("Exercise_6.pdf")

# Exercise 7
Compute expansion coefficients of the Green’s function of the Hubbard atom for lower temperatures. Check how the number of basis functions scales with respect to $\beta$.
For $0<\tau<\beta$,
$$
G(\tau) = \sum_{l=0}^{\infty}\frac{\sqrt{2l+1}}{\beta}P_l(x(\tau))G_l
$$
$$
G_l = \sqrt{2l+1}\int_0^{\beta}\mathrm{d}\tau P_l(x(\tau))G(\tau)
$$

In [234]:
deg = 100
lvec = [(i-1) for i in 1:deg]
P_l  = [SpecialPolynomials.basis(Legendre,l-1) for l in 1:deg]
xs = collect(range(-1,1,length=1000))
plt.figure()
for l in 1:3
    plt.plot(xs,P_l[l].(xs),label="l=$(l-1)")
end
plt.legend(loc="upper right")
plt.savefig("check_legendre_basis.pdf")

In [235]:
println(typeof(P_l[1]))
println(size(P_l[1]))
legg_val = zeros(Float64,length(x),deg)
for l in 1:deg
    legg_val[:,l] = P_l[l].(xs)
end
legg_val
plt.figure()
for l in 1:3
    plt.plot(xs,legg_val[:,l],label="l=$(l-1)")
end
plt.xlabel("x")
plt.ylabel("P_l(x)")
plt.legend(loc="upper right")
plt.savefig("check_legendre_basis2.pdf")


Legendre{Float64,:x,1}
(1,)


In [236]:
using OMEinsum,Einsum

In [237]:
U = 1.
β = 300.
# set Gauss-Legendre quadrature
deg = 100
x, w = gausslegendre(deg)
τs_leggauss = (x .+ 1)*β/2
ws_leggauss = (β/2) * w

lvec = [(l-1) for l in 1:deg]
P_l  = [SpecialPolynomials.basis(Legendre,l-1) for l in 1:deg]
legg_val = zeros(Float64,deg,deg)
for l in 1:deg
    legg_val[:,l] = P_l[l].(x)
end

G_tau = G.(τs_leggauss,U,β)

G_l = zeros(Complex,deg)
@einsum G_l[l] = sqrt(2*lvec[l]+1) * legg_val[t,l] * G_tau[t] * ws_leggauss[t]

plt.figure()
plt.semilogy(lvec,abs.(G_l),ls=" ",marker="x")
plt.xlabel("l")
plt.ylabel("|G_l|")
plt.savefig("Exercise_7.pdf")

# Exercise 7.5
reconstruct $G(i\nu)$ from the expansion coefficients in the Legendre basis for the Hubbard atom with $U=1$ and $\beta = 300.$
We obtain Matsubara Green' function given by
$$
G(i\nu) = \sum_{l=0}^{\infty}T_{nl}G_l,
$$
where 
$$
T_{nl} = \frac{\sqrt{2l+1}}{\beta}\int_0^{\beta}\mathrm{d}\tau e^{i\nu\tau}P_l(x(\tau)) 
       = (-1)^ni^{l+1}\sqrt{2l+1}j_l(n\pi/2)
$$

In [238]:
using SpecialFunctions

In [239]:
U = 1.
β = 300.
T = 1/β
# set Gauss-Legendre quadrature
deg = 100
x, w = gausslegendre(deg)
τs_leggauss = (x .+ 1)*β/2
ws_leggauss = (β/2) * w

lvec = [(l-1) for l in 1:deg]
P_l  = [SpecialPolynomials.basis(Legendre,l-1) for l in 1:deg]
legg_val = zeros(Float64,deg,deg)
for l in 1:deg
    legg_val[:,l] = P_l[l].(x)
end

n_max = 2000
step_ν = 20
νs = [(2n+1)*π*T for n in -n_max:step_ν:n_max]
T_nl = zeros(Complex,length(νs),deg)

for i_l in 1:deg
    i_l = lvec[i_l]
    for j_n in 1:length(νs)
        j_ν = νs[j_n]
        T_nl[j_n,i_l+1] = (-1)^(((1 + step_ν*(j_n-1)))-(n_max+1)) * im^(i_l+1) * sqrt(2*i_l+1) * sphericalbesselj(i_l,abs(j_ν / (π*T)) * π/2)
    end
end

G_iν_reconst =  T_nl * G_l

# Exact
G_iν_exact = G.(im .* νs,U)

plt.figure()
plt.plot(νs,imag.(G_iν_reconst),ls="-",marker="x",label="Reconstructed")
plt.plot(νs,imag.(G_iν_exact)  ,ls="-",marker="o",label="Exact")
plt.xlabel("ν")
plt.ylabel("Im G(iν)")
plt.xlim(-40,40)
plt.ylim(-1,1)
plt.legend(loc="upper right")
plt.savefig("Exercise_7.5_1.pdf")

In [240]:
plt.figure()
plt.semilogy(νs,abs.(G_iν_reconst .- G_iν_exact),label="Absolute error")
plt.semilogy(νs,abs.((G_iν_reconst .- G_iν_exact) ./ G_iν_exact),ls="-",label="Relative error")
plt.xlabel("ν")
plt.ylabel("Error")
plt.xlim(-40,40)
plt.legend(loc="upper right")
plt.savefig("Exercise_7.5_2.pdf")

# Exercise 8
$$
G(\tau) = - \int_{}^{}\mathrm{d}\omega K^{\mathrm{F}}(\tau,\omega)\rho(\omega),
$$
where
$$
K^{\mathrm{F}}(\tau,\omega) = \frac{e^{-\tau\omega}}{1+e^{-\beta\omega}},
$$
$$
G(\tau_i) = -\frac{2\omega_{\mathrm{max}}}{N_\omega} \sum_j K(\tau_i,\omega_j)\rho(\omega_j).
$$

In [241]:
function gaussian(x,μ,σ)
    (1/(sqrt(2π)*σ)) * exp(-((x-μ)/σ)^2)
end

ρ = ω -> 0.2 * gaussian(ω,0.0,0.15) + 0.4 * gaussian(ω,1.0,0.8) + 0.4 * gaussian(ω,-1.0,0.8)
ωs = collect(range(-2.5,2.5,length=1000))
plt.figure()
plt.plot(ωs,ρ.(ωs))
plt.xlabel("ω")
plt.ylabel("ρ(ω)")
plt.savefig("check_spectural_function.pdf")

In [242]:
function kernel_f(τ,ω,β)
    exp(-τ*ω) / (1+exp(-β*ω))
end

β = 10.0
ω_max = 10.0
N_ω = 1000
N_τ = 1000
ωs = collect(range(-ω_max,ω_max,length=N_ω))
τs = collect(range(0,β,length=N_τ))
K_tau_omega = zeros(Float64,N_τ,N_ω)
for j_ω in 1:N_ω
    for i_τ in 1:N_τ
        K_tau_omega[i_τ,j_ω] = kernel_f(τs[i_τ],ωs[j_ω],β)
    end
end

ρ_ω = ρ.(ωs)
G_tau = zeros(Float64,N_τ)
@einsum G_tau[i] = - (2ω_max/N_ω) * K_tau_omega[i,j] * ρ_ω[j]

plt.figure()
plt.plot(τs,G_tau)
plt.xlabel("τ")
plt.ylabel("G(τ)")
plt.ylim(-0.6,0)
plt.grid()
plt.savefig("check_g_tau.pdf")


Reconstruct $\rho(\omega)$
$$
\rho(\omega_j) = -\sum_i(K^{-1})_{ji}G(\tau_i)
$$
where $K^{-1}$ is inverse matrix of $K$ given by
$$
(K)_{ij} = \frac{2\omega_{\mathrm{max}}}{N_{\omega}}K(\tau_i,\omega_i).
$$

In [243]:
ρ_ω_reconst =  zeros(Float64,N_ω)
K_ij = (2ω_max/N_ω) .* K_tau_omega
K_inv = pinv(K_ij,rtol=1e-3)
@einsum ρ_ω_reconst[j] =  - K_inv[j,i] * G_tau[i]

plt.figure()
plt.plot(ωs,ρ.(ωs),label="Original")
plt.plot(ωs,ρ_ω_reconst,label="Reconstructed")
plt.xlabel("ω")
plt.ylabel("ρ(ω)")
plt.grid()
plt.legend(loc="upper right")
plt.savefig("check_reconstructed_rho.pdf")

#G_tau_reconst = zeros(Float64,N_τ)
#@einsum G_tau_reconst[i] = - K_ij[i,j] * ρ_ω_reconst[j]
G_tau_reconst = - K_ij * ρ_ω_reconst
plt.figure()
plt.semilogy(τs,-G_tau,label="Original")
plt.semilogy(τs[1:10:end],-G_tau_reconst[1:10:end],label="Reconstructed",ls="-",marker="x")
plt.semilogy(τs,abs.(G_tau .- G_tau_reconst),label="Difference")
plt.xlabel("τ")
plt.ylabel("G(τ)")
plt.grid()
plt.legend(loc="center")
plt.savefig("check_reconstructed_gtau.pdf")

In [244]:
plt.figure()
plt.plot(τs,abs.(G_tau .- G_tau_reconst),ls="-",marker="x")
plt.xlabel("τ")
plt.ylabel("Error in G(τ)")
plt.savefig("check_error_in_gtau.pdf")

we use singular value decompositon of K
$$
\bm{K} = \bm{U}\bm{S}\bm{V^{\dagger}}.
$$
where $\bm{U},\bm{V}$ are matrices consisteng of singular vectors and $\bm{S}$ is diagonal matrix consisting of the corresponding singular values $s_i(\ge0).$

In [245]:
u,s,v = svd(K_ij)
@assert K_ij ≈ u * Diagonal(s) * v' # ' operator denotes Hermite conjugate

plt.figure()
plt.semilogy(s[1:30],ls="",marker="x")
plt.xlabel("i")
plt.ylabel("si")
plt.savefig("check_singular_values.pdf")

In [246]:
num_sv = 4
plt.figure()
for i in 1:num_sv
    plt.plot(τs,u[:,i],label="i = $(i)")
end
plt.xlabel("τ")
plt.ylabel("u_i")
plt.tight_layout()
plt.legend(loc="upper right")
plt.savefig("check_u.pdf")

plt.figure()
for i in 1:num_sv
    plt.plot(ωs,v[:,i],label="i = $(i)")
end
plt.xlabel("ω")
plt.ylabel("v_i")
plt.legend(loc="upper right")
plt.tight_layout()
plt.savefig("check_v.pdf")



We obtain original and reconstructed $G(\tau)$/$\rho(\omega)$ projected onto $u_i$/$v_i$
$$
g_i = u_i^{\dagger}\bm{g} ,\ \rho_i = v_i^{\dagger}\bm{\rho},
$$

In [247]:
g_i_original = u' * G_tau
g_i_reconst  = u' * G_tau_reconst
i = [i for i in 1:length(g_i_original)]
plt.figure()
plt.semilogy(i[1:2:end],abs.(g_i_original[1:2:end]),ls="",marker="+",label="Original")
plt.semilogy(i[1:2:end],abs.(g_i_reconst[1:2:end]), ls="",marker="x",label="Reconstructed")
plt.xlabel("i")
plt.ylabel("|g_i|")
plt.xlim(0,30)
plt.ylim(1e-10,10)
plt.savefig("check_g_i.pdf")

In [248]:
ρ_i_original = v' * ρ_ω
ρ_i_reconst  = v' * ρ_ω_reconst
i = [i for i in 1:length(ρ_i_original)]
plt.figure()
plt.semilogy(i[1:2:end],abs.(ρ_i_original[1:2:end]),ls="",marker="+",label="Original")
plt.semilogy(i[1:2:end],abs.(ρ_i_reconst[1:2:end]), ls="",marker="x",label="Reconstructed")
plt.xlabel("i")
plt.ylabel("|ρ_i|")
plt.xlim(0,30)
plt.ylim(1e-3,10)
plt.savefig("check_rho_i.pdf")

# Exercise 9
We expand some spectral functions $\rho(\omega)$ in $V_l(\omega)$ using
$$
\rho_l = \int_{\omega_{\mathrm{max}}}^{\omega_{\mathrm{max}}}\mathrm{d}\omega V_l(\omega)\rho(\omega),
$$
and compute the expansion coefficients of $G(\tau)$ using
$$
G_l = - S_l \rho_l.
$$
Then evaluate $G(\tau)$ on a uniform $\tau$ mesh.

In [249]:
using PyCall 
irbasis3 = pyimport("irbasis3")

PyObject <module 'irbasis3' from '/Users/humiyakakizawa/.pyenv/versions/3.9.0/lib/python3.9/site-packages/irbasis3/__init__.py'>

In [250]:
lambda_ = 100
beta = 10
K = irbasis3.KernelFFlat(lambda_=lambda_)
basis = irbasis3.FiniteTempBasis(K,statistics="F",beta=beta,eps=1e-10)

PyObject <irbasis3.basis.FiniteTempBasis object at 0x17936b490>

In [251]:
# check S,U,V
plt.figure()
plt.semilogy(basis.s,ls="",marker="o")
plt.xlabel("l")
plt.ylabel("S_l")
plt.savefig("check_irbasis_s.pdf")

N_τ = 1000
τs = collect(range(0,beta,length=N_τ))
max_l = 4
plt.figure()
for l in 1:max_l
    plt.plot(τs,basis.u[l].(τs),label="l = $(l)")
end
plt.xlabel("τ")
plt.ylabel("U_l(τ)")
plt.legend(loc="upper right")
plt.savefig("check_irbasis_u_1.pdf")


N_ω = 1000
ω_max = 10
ωs = collect(range(-ω_max,ω_max,length=N_ω))
max_l = 4
plt.figure()
for l in 1:max_l
    plt.plot(ωs,basis.v[l].(ωs),label="l = $(l)")
end
plt.xlabel("ω")
plt.ylabel("V_l(τ)")
plt.legend(loc="upper right")
plt.savefig("check_irbasis_v.pdf")


In [252]:
# We choose the superposition of three gausian distributions as a spectral functoin 
function gaussian(x,μ,σ)
    (1/(sqrt(2π)*σ)) * exp(-((x-μ)/σ)^2)
end

ρ = ω -> 0.2 * gaussian(ω,0.0,0.15) + 0.4 * gaussian(ω,1.0,0.8) + 0.4 * gaussian(ω,-1.0,0.8)
plt.figure()
plt.plot(ωs,ρ.(ωs))
plt.xlabel("ω")
plt.ylabel("ρ(ω)")
plt.xlim(-5,5)
plt.savefig("check_rho_omega.pdf")

ρ_l = basis.v.overlap(ρ)
#ls = [l for l in 1:length(basis.s)]
#plt.figure()
#=
plt.semilogy(1:2:end,1,ls="",marker="o")
plt.xlabel("l")
plt.ylabel("|ρ_l|")
plt.savefig("check_rho_l.pdf")
=#

PyCall.PyJlError: PyCall.PyJlError(MethodError(-, ([-9.999789856040653, -9.998894919128139, -9.997293631124608, -9.99500058323833, -9.992037140805982, -9.988430962065898, -9.984215713370574, -9.979430749890506, -9.974120746842072, -9.968335281925054, -9.962128372256252, -9.955557969950235, -9.948685420999437, -9.941574892484065, -9.93429277345069, -9.926907055049195, -9.919486695713491, -9.912100977311997, -9.904818858278622, -9.89770832976325, -9.890835780812452, -9.884265378506434, -9.878058468837633, -9.872273003920615, -9.86696300087218, -9.862178037392113, -9.857962788696788, -9.854356609956705, -9.851393167524357, -9.849100119638079, -9.847498831634548, -9.846603894722033, -9.845936484130197, -9.843989129614062, -9.840504777385764, -9.835515177454624, -9.829066820119891, -9.821219888836476, -9.81204764052636, -9.801635710800202, -9.790081311520808, -9.777492322200393, -9.763986282382657, -9.749689294042447, -9.734734844125873, -9.719262558177123, -9.703416896669173, -9.6873458062

In [253]:
g_l = - basis.s .* ρ_l

UndefVarError: UndefVarError: ρ_l not defined

# Exercise 10


# Playground

In [257]:
using FastGaussQuadrature,SpecialPolynomials

In [259]:
x, w = gausslegendre(3)

([-0.7745966692414834, 0.0, 0.7745966692414834], [0.5555555555555556, 0.8888888888888888, 0.5555555555555556])

In [260]:
myfunc(x) = x^4
I = dot(w,myfunc.(x))
@assert I ≈ 2/5
p3 = SpecialPolynomials.basis(Legendre,3)
x2 = collect(range(-1,1,length=1000))

plt.figure()
plt.plot(x2,p3.(x2),label="P_3")
plt.plot(x,[0,0,0],marker="o")
plt.xlabel("x")
plt.ylabel("3rd basis of Legendre polynomials")
plt.savefig("check_gaussquad.pdf")

In [261]:
x = collect(range(0,9,length=10))
y = exp.(x)
f = LinearInterpolation(x,y)

xnew = collect(range(0,9,length=10000))
ynew = f(xnew)

plt.figure()
plt.plot(x,y,marker="o")
plt.plot(xnew,ynew)
plt.savefig("check_interpolate.pdf")